In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file
import matplotlib.pyplot as plt
import seaborn as sns
import ast   # For converting string to list
import nltk   #Natural Language Toolkit

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', None)
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import dataset
movies = pd.read_csv('F:\ML Projects\movie reccomendation project\Hollywood movie recommendation with Flask/tmdb_5000_movies.csv') #movie names ,ratings
credit = pd.read_csv('F:\ML Projects\movie reccomendation project\Hollywood movie recommendation with Flask/tmdb_5000_credits.csv') #cast ,crew

movies.head(5)
credit.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
# Merge movie and credit dataset
movies_df = movies.merge(credit, on='title')
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
# Distribution of Original_language
movies_df['original_language'].value_counts()

en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

# dropping some columns which are not useful 

In [6]:
# Filter relevent features
movies_final = movies_df[['id','title','genres','keywords', 'overview',  'cast', 'crew']]
movies_final.head()

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
# Check missing values
movies_final.isna().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [8]:
# 'Overview' feature have 3 missing values. It is not a big number. So, we drop them

In [9]:
# Drop missing values
movies_final.dropna(inplace=True)

In [10]:
# Check for duplicates 
movies_final.duplicated().sum()

0

In [11]:
movies_final.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
# convert strings  ({"id": 12, "name": "Adventure"}) to list
def convert(string):
    listt = []  # List to store name
    for i in ast.literal_eval(string):   #ast.literal_eval() convert str into list
        listt.append(i['name'])
    return listt

In [13]:
# Apply convert function to genres
movies_final['genres'] = movies_final['genres'].apply(convert)

In [14]:
# Apply convert function to keywords
movies_final['keywords'] = movies_final['keywords'].apply(convert)

In [15]:
# Function to extract 3 names
def convert2(string):
    list2 =[]
    counter =0
    for i in ast.literal_eval(string):
        if counter !=3:   # Limit 3 only
            list2.append(i['name'])
            counter +=1
        else:
            break
    return list2

In [16]:
# Apply convert3 function
movies_final['cast'] = movies_final['cast'].apply(convert2)

In [17]:
# Function to extract director's name
def extract_director(string):
    list3 = []
    for i in ast.literal_eval(string):
        if i['job'] == 'Director':
            list3.append(i['name'])
            break
    return list3

In [18]:
# Apply extract_director function
movies_final['crew'] = movies_final['crew'].apply(extract_director)

In [19]:
# String to list 
movies_final['overview'] = movies_final['overview'].apply(lambda x: x.split())

In [20]:
# Removing spaces between words
movies_final['genres'] = movies_final['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_final['keywords'] = movies_final['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies_final['cast'] = movies_final['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies_final['crew'] = movies_final['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [21]:
# We have removed the spaces between words. Sometime, we could get double words of same name. At that time, our model may get confused and recommed wrong movie.

In [22]:
movies_final.head(10)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]
5,559,Spider-Man 3,"[Fantasy, Action, Adventure]","[dualidentity, amnesia, sandstorm, loveofone's...","[The, seemingly, invincible, Spider-Man, goes,...","[TobeyMaguire, KirstenDunst, JamesFranco]",[SamRaimi]
6,38757,Tangled,"[Animation, Family]","[hostage, magic, horse, fairytale, musical, pr...","[When, the, kingdom's, most, wanted-and, most,...","[ZacharyLevi, MandyMoore, DonnaMurphy]",[ByronHoward]
7,99861,Avengers: Age of Ultron,"[Action, Adventure, ScienceFiction]","[marvelcomic, sequel, superhero, basedoncomicb...","[When, Tony, Stark, tries, to, jumpstart, a, d...","[RobertDowneyJr., ChrisHemsworth, MarkRuffalo]",[JossWhedon]
8,767,Harry Potter and the Half-Blood Prince,"[Adventure, Fantasy, Family]","[witch, magic, broom, schoolofwitchcraft, wiza...","[As, Harry, begins, his, sixth, year, at, Hogw...","[DanielRadcliffe, RupertGrint, EmmaWatson]",[DavidYates]
9,209112,Batman v Superman: Dawn of Justice,"[Action, Adventure, Fantasy]","[dccomics, vigilante, superhero, basedoncomicb...","[Fearing, the, actions, of, a, god-like, Super...","[BenAffleck, HenryCavill, GalGadot]",[ZackSnyder]


In [23]:
# Create new feature named 'tags'
movies_final['tags'] = movies_final['genres'] + movies_final['overview'] + movies_final['keywords'] + movies_final['cast'] + movies_final['crew']

In [24]:
# Select required features only

movies_final.rename(columns={'id': 'movie_id'}, inplace=True)

movies = movies_final[['movie_id', 'title', 'tags']]

movies.head()

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, John, Cart..."


In [25]:
# Convert list to string
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [26]:
# Convert to lower case
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [27]:
movies.head(5)

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction in the...
1,285,Pirates of the Caribbean: At World's End,"adventure fantasy action captain barbossa, lon..."
2,206647,Spectre,action adventure crime a cryptic message from ...
3,49026,The Dark Knight Rises,action crime drama thriller following the deat...
4,49529,John Carter,action adventure sciencefiction john carter is...


In [28]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

The PorterStemmer is an algorithm for stemming English words, developed by Martin Porter in 1979. It applies a set of rules to reduce different inflected or derived words to their base or root form. For example:

"running" --> "run"
"jumps" --> "jump"
"happiness" --> "happi"
"going" --> "go"
Stemming is helpful in NLP tasks like information retrieval, text mining, and search engines, where it allows words with the same root to be treated as the same word. This can help in reducing the dimensionality of the data and improving the efficiency of certain NLP algorithms.

In [29]:
#
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [30]:
movies['tags'] = movies['tags'].apply(stem)

In [31]:
movies.head()

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict in the 22n...
1,285,Pirates of the Caribbean: At World's End,"adventur fantasi action captain barbossa, long..."
2,206647,Spectre,action adventur crime a cryptic messag from bo...
3,49026,The Dark Knight Rises,action crime drama thriller follow the death o...
4,49529,John Carter,action adventur sciencefict john carter is a w...


 CountVectorizer class from scikit-learn (sklearn) to convert a collection of text documents (in this case, the 'tags' column of the DataFrame df) into a matrix of token counts. This process is commonly known as "Bag-of-Words" representation.

In [32]:
# Convert the string to vector format
count_vect = CountVectorizer(max_features=5000, stop_words='english')
# Fit the dataset to the transformer
vectors = count_vect.fit_transform(movies['tags']).toarray()

In [33]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

 Cosine distance is a metric that measures the similarity between two vectors in a multi-dimensional space. It is often used to compare the similarity between documents or text representations.

In [34]:
# Cosine distance between vectors
from sklearn.metrics.pairwise import cosine_similarity


In [35]:
similarity = cosine_similarity(vectors)

In [36]:
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [37]:
len(similarity[0])

4806

# Recommendation

In [38]:
def recommend(movie):
    index = movies[movies['title']==movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key = lambda x:x[1])
    
    for i in distances[1:6]:
        print(movies.iloc[i[0]].title)

In [39]:
movies[movies['title']=='Avatar'].index[0]

0

In [40]:
sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x:x[1])

[(0, 1.0000000000000002),
 (1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574),
 (582, 0.24511108480187255),
 (1204, 0.24455799402225925),
 (1194, 0.2367785320221084),
 (61, 0.23179316248638276),
 (778, 0.23174488732966073),
 (4048, 0.2278389747471728),
 (1920, 0.2252817784447915),
 (2786, 0.21853668936906193),
 (172, 0.21239769762143662),
 (972, 0.2108663315950723),
 (322, 0.2105263157894737),
 (2333, 0.20443988269091456),
 (3608, 0.20437977982832192),
 (260, 0.20395079136182276),
 (151, 0.2029530274475215),
 (4192, 0.2029530274475215),
 (1444, 0.20277677641345318),
 (74, 0.2024645717996314),
 (1089, 0.2020475485519274),
 (3675, 0.1979082783981174),
 (973, 0.19767387315371682),
 (577, 0.1976738731537168),
 (47, 0.19672236884115843),
 (2971, 0.1925214071641298),
 (942, 0.19134594929397597),
 (495, 0.19088542889273336),
 (1201, 0.19088542889273336),
 (305, 0.19007487139298027),
 (4, 0.189299409712

In [41]:
recommend("Avatar")

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [42]:
import pickle


In [43]:
pickle.dump(movies, open('model/movies_list.pkl ','wb'))


In [44]:
pickle.dump(similarity, open('model/similarity.pkl ','wb'))


In [45]:
movies = pickle.load(open('model/movies_list.pkl', 'rb'))

In [46]:
movies

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict in the 22n...
1,285,Pirates of the Caribbean: At World's End,"adventur fantasi action captain barbossa, long..."
2,206647,Spectre,action adventur crime a cryptic messag from bo...
3,49026,The Dark Knight Rises,action crime drama thriller follow the death o...
4,49529,John Carter,action adventur sciencefict john carter is a w...
...,...,...,...
4804,9367,El Mariachi,action crime thriller el mariachi just want to...
4805,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is up...
4806,231617,"Signed, Sealed, Delivered","comedi drama romanc tvmovi ""signed, sealed, de..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...
